In [7]:
try:
    import selenium
except ModuleNotFoundError:
    %pip install selenium

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def get_value(element):
    value = element.get_attribute("value")
    if value == "-1":
        return None
    return value

# Set up WebDriver
driver = webdriver.Chrome()

# Navigate to the website and manually solve the CAPTCHA
driver.get("https://airquality.cpcb.gov.in/AQI_India/")

> At this point, go to the browser window and solve the CAPTCHA manually.

In [5]:
# Select state
state_dropdown = Select(driver.find_element(By.ID, "states"))

data_list = []
pbar = tqdm(total=1000)
for state_element in state_dropdown.options:
    state_name = get_value(state_element)
    if state_name is None:
        continue
    state_dropdown.select_by_value(state_name)
    sleep(0.1)
    city_dropdown = Select(driver.find_element(By.ID, "cities"))
    for city_element in city_dropdown.options:
        city_name = get_value(city_element)
        if city_name is None:
            continue
        city_dropdown.select_by_value(city_name)
        station_dropdown = Select(driver.find_element(By.ID, "stations"))
        sleep(0.1)
        for station_element in station_dropdown.options:
            site_id = get_value(station_element)
            if site_id is None:
                continue
            station_name = station_element.text
            data_list.append((state_name, city_name, station_name, site_id))
            pbar.update(1)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [6]:
len(data_list)

538

In [9]:
df = pd.DataFrame(data_list, columns=["state", "city", "station", "site_id"])
df.head(2)

,state,city,station,site_id
0,Andhra Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",site_1406
1,Andhra Pradesh,Anantapur,"Gulzarpet, Anantapur - APPCB",site_5632


In [10]:
df.to_csv("site_ids.csv", index=None)